# Journal de bord

In [7]:
#Importation des bibli pour la suite
import numpy as np

## Mercredi 10 Février

idée projet : 
- simulation foule/ gaz/liquide

- lien entre grandeur thermo (S,U...) et "classique" (x,v,a...)

- Faire comparaison T_eq avec modele Thermo / Thermostat

- trouver état d'équilibre comment le definir ?

- calculer grandeur thermo comment calculer entropie ?

- A partir de combien de particule on tend bien vers température d'équilibre thermo ?

- L'état d'équilibre depend des conditions initiales ?


### Réalisation

- Définir bords

- (Défnir interaction simple entre molécules)

- Définir interaction particules / mur dans un premier temps. Type loi de Descartes ?

- utilisation d'objet ?

## Mercredi 24 Février

Changement : on essaie de modéliser une salle avec un mouvement de panique : en combien de temps les personnes parviennent-elles à sortir ? Quel est la géométrie optimale ?

Première idée de code :

En fait on part sur une programmation objet. Il faut coder objet personne et objet mur et objet sortie. La sortie applique une force aux personnes constante et en ligne droite. Chaque personne applique sur toutes les autres une force (decroissante en fonction de la distance). On a une matrice d'interaction : force entre chaque personne. On n'a besoin que de la partie triangulaire superieure (juste force opposée sur la partie inférieure).

In [1]:
class personne:
    """
    caractéristiques :
    position x,y
    vitesse vx,vy
    (mouvement oui ou non)
    rayon r
    masse m
    """
    def __init__(self,x,y,vx,vy,r,m):
        """Définition des parametre initiaux"""
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.r = r
        self.m = m
        
    def distance(self,autre):
        """Retourne la distance entre la particule et une autre"""
        dx = self.x - autre.x
        dy = self.y - autre.y
        return np.sqrt(dx**2+dy**2)
    
    def superpose(self,autre):
        """Retourne True si les deux particules se superposent"""
        answer = distance(self,autre) < self.r + autre.r
        return answer
        
    def force(self,autre):
        
        

In [11]:
def distance(personneA,personneB):
    """retourne la distance entre deux personne : personneA et personneB"""
    dx = personneB.x - personneA.x
    dy = personneB.y - personneA.y
    return np.sqrt(dx**2+dy**2)

In [12]:
def force(personneA,personneB):
    """Retourne la force (répulsive) qu'applique la personneA sur la personneB"""
    F = -1/distance(personneA,personneB)**7

## Mercredi 3 mars

Nous continuons de definir la classe personne. Ajout de distance et superpose. Nous continuons de comprendre l'animation.
Nous devons dans un premier temps tester notre code sur 3 particules : pour voir si ça marche.